In [1]:
!python  -V

Python 3.9.12


In [2]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error

import pickle

## Load Data (Parquet)


In [3]:
data = pd.read_parquet("./data/green_tripdata_2021-01.parquet")

In [4]:
data.head(10)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.5,0.5,0.5,0.00,0.0,None,0.3,6.80,2.0,1.0,0.00
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.0,0.5,0.5,2.81,0.0,None,0.3,16.86,1.0,1.0,2.75
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.0,0.5,0.5,1.00,0.0,None,0.3,8.30,1.0,1.0,0.00
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.0,0.5,0.5,0.00,0.0,None,0.3,9.30,2.0,1.0,0.00
4,2,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2.0,265,265,3.0,0.00,-52.0,0.0,-0.5,0.00,0.0,None,-0.3,-52.80,3.0,1.0,0.00
5,2,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2.0,265,265,3.0,0.00,52.0,0.0,0.5,0.00,0.0,None,0.3,52.80,2.0,1.0,0.00
6,2,2021-01-01 00:19:14,2021-01-01 00:19:21,N,5.0,265,265,1.0,0.00,180.0,0.0,0.0,36.06,0.0,None,0.3,216.36,1.0,2.0,0.00
7,2,2021-01-01 00:26:31,2021-01-01 00:28:50,N,1.0,75,75,6.0,0.45,3.5,0.5,0.5,0.96,0.0,None,0.3,5.76,1.0,1.0,0.00
8,2,2021-01-01 00:57:46,2021-01-01 00:57:57,N,1.0,225,225,1.0,0.00,2.5,0.5,0.5,0.00,0.0,None,0.3,3.80,2.0,1.0,0.00
9,2,2021-01-01 00:58:32,2021-01-01 01:32:34,N,1.0,225,265,1.0,12.19,38.0,0.5,0.5,2.75,0.0,None,0.3,42.05,1.0,1.0,0.00


## Feature Engineering

In [5]:
# Calculate Trip-time
data["duration"] = data.lpep_dropoff_datetime - data.lpep_pickup_datetime

# convert to minutes
data.duration = data.duration.apply(lambda dur: dur.total_seconds()/60)

# focus on data that is less than 60min and more than 1min
data = data[(data.duration >= 1) & (data.duration <= 60)]

# Take a subset
# data = data[data.trip_type == 2]

In [6]:
# which variables to take?
categorical = ["PULocationID", "DOLocationID"]
numerical = ["duration"]

data = data[categorical + numerical]

# do one hot encoding for categorical features
data[categorical] = data[categorical].astype(str)

In [7]:
# convert df to dictionaries
train_dicts = data.to_dict(orient="records")

dv = DictVectorizer()
x_train = dv.fit_transform(train_dicts)

In [8]:
# Set the Target
target = "duration"
y_train = data[target].values

## Create a function

In [9]:
def process_data(filename):
    data = pd.read_parquet(filename)
    
    data["duration"] = data.lpep_dropoff_datetime - data.lpep_pickup_datetime
    data.duration = data.duration.apply(lambda dur: dur.total_seconds()/60)
    data = data[(data.duration >= 1) & (data.duration <= 60)]
    
    categorical = ["PULocationID", "DOLocationID"]

    # do one hot encoding for categorical features
    data[categorical] = data[categorical].astype(str)
    
    return data

In [10]:
# Load train and validation data
train_path = "./data/green_tripdata_2021-01.parquet"
train_d = process_data(train_path)

val_path = "./data/green_tripdata_2021-02.parquet"
val_d = process_data(val_path)

In [11]:
train_d.shape, val_d.shape

((73908, 21), (61921, 21))

In [12]:
# create new feature
train_d["PU_DO"] = train_d["PULocationID"] + "-" + train_d["DOLocationID"]
val_d["PU_DO"] = val_d["PULocationID"] + "-" + val_d["DOLocationID"]

In [13]:
# create DictVectorizer
dv = DictVectorizer()
categorical = ["PU_DO"]
numerical = ["trip_distance"]

train_dicts = train_d[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = val_d[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)

In [14]:
target = "duration"
Y_train = train_d[target].values
Y_val = val_d[target].values

## Training + Validation

In [15]:
with mlflow.start_run():

    mlflow.set_tag("developer", "davidweber")

    mlflow.log_param("train-data-path", train_path)
    mlflow.log_param("validation-data-path", val_path)

    #alpha = 0.01
    #mlflow.log_param("alpha", alpha)

    lr = LinearRegression()

    mlflow.log_param("model", lr)
    lr.fit(X_train, Y_train)

    Y_pred = lr.predict(X_val)

    rmse = mean_squared_error(Y_val, Y_pred, squared=False)
    print(rmse)
    mlflow.log_metric("rmse", rmse)

7.479603461126994


In [16]:
# Save Model
with open("./models/lr-reg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)

In [17]:
with mlflow.start_run():

    mlflow.set_tag("developer", "davidweber")

    mlflow.log_param("train-data-path", train_path)
    mlflow.log_param("validation-data-path", val_path)

    alpha = 0.01
    mlflow.log_param("alpha", alpha)

    model = Lasso(alpha=alpha)

    mlflow.log_param("model", model)
    model.fit(X_train, Y_train)

    Y_pred = lr.predict(X_val)

    rmse = mean_squared_error(Y_val, Y_pred, squared=False)
    print(rmse)
    mlflow.log_metric("rmse", rmse)

7.479603461126994


## XGBoost with Hyperparam Optimization

In [18]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

/Users/da.weber/opt/anaconda3/envs/mlops/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [19]:
train = xgb.DMatrix(X_train, label=Y_train)
valid = xgb.DMatrix(X_val, label=Y_val)

In [20]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(Y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [36]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[22:29:19] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:7.07373                           
[1]	validation-rmse:6.70087                           
[2]	validation-rmse:6.66029                           
[3]	validation-rmse:6.64676                           
[4]	validation-rmse:6.63224                           
[5]	validation-rmse:6.62558                           
[6]	validation-rmse:6.62106                           
[7]	validation-rmse:6.61015                           
[8]	validation-rmse:6.60685                           
[9]	validation-rmse:6.60322                           
[10]	validation-rmse:6.60142                          
[11]	validation-rmse:6.59924                          
[12]	validation-rmse:6.59223                          
[13]	validation-rmse:6.58962                          
[14]	validation-rmse:6.59012                          
[15]	validation-rmse:6.58941                          
[

In [21]:
mlflow.xgboost.autolog(disable=True)

In [22]:
# train the best model
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=Y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    Y_pred = booster.predict(valid)
    rmse = mean_squared_error(Y_val, Y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    #save dict-vectorizer
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    
    # log dict-vect as an artifact
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    # log xgboost model
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[12:00:44] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48427
[1]	validation-rmse:17.95632
[2]	validation-rmse:16.59113
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29011
[5]	validation-rmse:13.32801
[6]	validation-rmse:12.47570
[7]	validation-rmse:11.72141
[8]	validation-rmse:11.05883
[9]	validation-rmse:10.47574
[10]	validation-rmse:9.96473
[11]	validation-rmse:9.51779
[12]	validation-rmse:9.12616
[13]	validation-rmse:8.78508
[14]	validation-rmse:8.49021
[15]	validation-rmse:8.23316
[16]	validation-rmse:8.01171
[17]	validation-rmse:7.81826
[18]	validation-rmse:7.65206
[19]	validation-rmse:7.50734
[20]	validation-rmse:7.38236
[21]	validation-rmse:7.27392
[22]	validation-rmse:7.17883
[23]	validation-rmse:7.09638
[24]	validation-rmse:7.02473
[25]	validation-rmse:6.96273
[26]	validation-rmse:6.90885
[27]	validation-rmse:6.86167
[28]	validation-rmse:6.82066
[29]	validation-rmse:6.78448
[30]	valida

/Users/da.weber/opt/anaconda3/envs/mlops/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## Loading the Model again, previously saved with MLFlow

In [23]:
logged_model = 'runs:/b28b262e25d54746afc299df2027c5d7/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Load model as xgboost Object
xgb_model = mlflow.xgboost.load_model(logged_model)

# Predict on a Pandas DataFrame.
# loaded_model.predict(pd.DataFrame(data))

xgb_model

[12:06:16] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[12:06:16] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


In [24]:
# Making predictions
preds = xgb_model.predict(valid)
preds[:10]

array([14.996554 ,  7.1544437, 14.240531 , 24.346413 ,  9.390426 ,
       17.119205 , 11.810911 ,  8.411136 ,  8.941674 , 20.334606 ],
      dtype=float32)